In [1]:
import sys
sys.path.append("..")
sys.path.append("../model")
from trajcl import TrajCL
from config import Config

In [2]:
conf = Config()

conf.dataset = 'generic_v3'
conf.post_value_updates()

In [3]:
model = TrajCL()

/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [4]:
model

TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTB(
      (pos_encoder): PositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
            )
            (linear1): Linear(in_features=256, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=256, bias=True)
            (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): SpatialMSM(


In [5]:
# from ah_databricks_data_loader import DatabricksDataLoader

# # Instantiate the DatabricksDataLoader.
# ddl = DatabricksDataLoader()

# # Load data.
# test_df_1 = ddl.load_as_spark(schema="datascience_scratchpad", table="la_traj_data_v9_part1")
# test_df_2 = ddl.load_as_spark(schema="datascience_scratchpad", table="la_traj_data_v9_part2")

:: loading settings :: url = jar:file:/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sagemaker-user/.ivy2/cache
The jars for the packages stored in: /home/sagemaker-user/.ivy2/jars
io.delta#delta-sharing-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-363156ba-5b88-47d0-814a-700990d74c79;1.0
	confs: [default]
	found io.delta#delta-sharing-spark_2.12;3.2.0 in central
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found io.delta#delta-sharing-client_2.12;1.0.5 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.2 in central
	found commons-codec#commons-codec;1.11 in central
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-o

In [5]:
# test_df= test_df_1.join(test_df_2, on  = ['userid','traj_date'])
# 

In [ ]:
# from pyspark.sql.functions import udf, col
# from pyspark.sql.types import StructType, StructField, ArrayType, DoubleType, DateType, TimestampType, IntegerType
# def get_weekday(traj_date):
#     return traj_date.weekday()

# import math
# def lonlat2meters(lon, lat):
#     semimajoraxis = 6378137.0
#     east = lon * 0.017453292519943295
#     north = lat * 0.017453292519943295
#     t = math.sin(north)
#     return semimajoraxis * east, 3189068.5 * math.log((1 + t) / (1 - t))
    
# get_weekday_udf = udf(lambda traj_date: get_weekday(traj_date), IntegerType())
# test_df = test_df.withColumn("weekday", get_weekday_udf(col("traj_date")))
# lonlat2meters_udf = udf(lambda traj: [list(lonlat2meters(p[0], p[1])) for p in traj], ArrayType(ArrayType(DoubleType())))
# test_df = test_df.withColumn("merc_seq", lonlat2meters_udf(col("polylines_final")))
# display(test_df)

In [ ]:
test_df.count()

In [ ]:
test_df.show(5)

In [ ]:
test_df.write.mode("overwrite").parquet("/home/sagemaker-user/TrajCL/data/la/la_subset_1k")

In [ ]:
# test_df.count()

In [6]:
from glob import glob
import pandas as pd
parquet_files = glob("/home/sagemaker-user/TrajCL/data/la/la_subset_1k/*.parquet")
total_count = 0
df_list = []
for file in parquet_files:
    df = pd.read_parquet(file)
    df_list.append(df)
    total_count += len(df)


print(f"Total records in repartitioned files: {total_count}")

Total records in repartitioned files: 39735


In [7]:
df_list[0].head()

,userid,traj_date,timestamps,polylines_final,employername,partition_id,pck_0_1,weekday,merc_seq
0,2844305,2025-06-24,"[2025-06-24T05:49:28.000000000, 2025-06-24T07:...","[[-118.39117477618277, 33.92325761467734], [-1...",Camping World/Gander Outdoors,2,1,1,"[[-13179245.29050212, 4018502.0477052485], [-1..."
1,2869661,2024-09-21,"[2024-09-21T08:54:19.000000000, 2024-09-21T11:...","[[-118.59423330634597, 34.1905517578125], [-11...",County Of Riverside,0,1,5,"[[-13201849.662681118, 4054417.2603058773], [-..."
2,2869661,2024-09-22,"[2024-09-22T07:02:57.000000000, 2024-09-22T11:...","[[-118.5940859861017, 34.190643310546875], [-1...",County Of Riverside,0,1,6,"[[-13201833.263066543, 4054429.58131514], [-13..."
3,2869661,2025-02-08,"[2025-02-08T10:03:51.000000000, 2025-02-08T10:...","[[-117.66765060248021, 34.03045269170906], [-1...",County Of Riverside,1,1,5,"[[-13098702.947908927, 4032891.811937815], [-1..."
4,2869661,2025-04-26,"[2025-04-26T09:22:08.000000000, 2025-04-26T09:...","[[-117.73360870444878, 34.025118917335035], [-...",County Of Riverside,1,1,5,"[[-13106045.37023376, 4032175.3819222637], [-1..."


In [7]:
# import pandas as pd
# test_df = pd.read_parquet("/home/sagemaker-user/TrajCL/data/parquet_files/test/nyc_df_v3_with_time/traj_test_df_v3_with_ts.parquet")

In [8]:
# userids = test_df['userid'].unique()

In [8]:
import torch
device = torch.device("cuda:0")
checkpoint_file = "/home/sagemaker-user/TrajCL/exp/generic_v3/generic_v3_TrajCL_best.pt"
checkpoint = torch.load(checkpoint_file)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

/tmp/ipykernel_6392/1038578970.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_file)


TrajCL(
  (clmodel): MoCo(
    (criterion): CrossEntropyLoss()
    (encoder_q): DualSTB(
      (pos_encoder): PositionalEncoding(
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (structural_attn): TransformerEncoder(
        (layers): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
            )
            (linear1): Linear(in_features=256, out_features=2048, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=2048, out_features=256, bias=True)
            (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (spatial_attn): SpatialMSM(


In [9]:
from utils.traj import *
import pickle

from torch.nn.utils.rnn import pad_sequence
embs = pickle.load(open("/home/sagemaker-user/TrajCL/data/generic_v3_cell250_embdim256_embs.pkl", 'rb')).to('cpu').detach() # tensor
cellspace = pickle.load(open("/home/sagemaker-user/TrajCL/data/generic_v3_cell250_cellspace.pkl", 'rb'))

max_batch_size = 512
def infer_batch(traj):
    traj_cell, traj_p = zip(*[merc2cell2(t, cellspace) for t in traj])
    # print(traj_cell)
    traj_emb_p = [torch.tensor(generate_spatial_features(t, cellspace)) for t in traj_p]
    traj_emb_p = pad_sequence(traj_emb_p, batch_first = False).to(device)
    traj_emb_cell = [embs[list(t)] for t in traj_cell]
    traj_emb_cell = pad_sequence(traj_emb_cell, batch_first = False).to(device)
    traj_len = torch.tensor(list(map(len, traj_cell)), dtype = torch.long, device = device)
    # time_indices = pad_sequence([torch.tensor(t, dtype=torch.long) for t in time_indices], batch_first=False, padding_value=-1).to(Config.device)
    # print(traj_emb_cell, traj_emb_p, traj_len)
    traj_embs = model.interpret(traj_emb_cell.float(), traj_emb_p.float(), traj_len, None)
    return traj_embs

def infer(traj):
    if len(traj)> max_batch_size:
        traj_embs = []
        for i in range(0, len(traj), max_batch_size):
            traj_batch = traj[i:i+max_batch_size]
            # time_indices_batch = time_indices[i:i+max_batch_size] 
            traj_embs.append(infer_batch(traj_batch))
        return torch.cat(traj_embs, dim=0)
    else:
        return infer_batch(traj)



/home/sagemaker-user/.conda/envs/trajcl/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [10]:
from sklearn.metrics.pairwise import cosine_similarity


In [11]:
unique_userids = pd.concat([df['userid'] for df in df_list]).unique()
print(f"Total unique userids: {len(unique_userids)}")

unique_employernames = pd.concat([df['employername'] for df in df_list]).unique()
print(f"Total unique employernames: {len(unique_employernames)}")

Total unique userids: 1000
Total unique employernames: 1007


In [12]:
def get_data_for_userid(userid):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    return df_user

def get_data_for_employername(df_user, employername):
    df_emp = df_user[df_user['employername']==employername].reset_index(drop=True)
    return df_emp


def get_data_for_partition(df_emp, partition):
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part = df_part[df_part['pck_0_1']>0].reset_index(drop=True) # filter out zero paycheck amount
    return df_part


def get_traj_and_time_data(df_part):
    traj = df_part['merc_seq'].values
    # time_indices = df_part['time_index_list'].values
    return traj

In [13]:
def apply_dbscan(embs, target_min_similarity=0.85):
    from sklearn.cluster import DBSCAN     # require >= 0.9 cosine similarity
    eps = 1.0 - target_min_similarity    # cosine distance threshold
    n_embs = embs.shape[0]
    db = DBSCAN(eps=eps, min_samples=max(int(n_embs*0.3), 5), metric="cosine", n_jobs=-1).fit(embs)
    return db

In [14]:
def get_min_max_xy(traj):
    min_x = min([p[0] for p in traj])
    max_x = max([p[0] for p in traj])
    min_y = min([p[1] for p in traj])
    max_y = max([p[1] for p in traj])
    return min_x, max_x, min_y, max_y

def calculate_dx_dy(k, target_min_x, target_min_y, min_x, max_x, min_y, max_y):
    dx = target_min_x+ (k - max_x - min_x) / 2
    dy = target_min_y+ (k - max_y - min_y) / 2
    return dx, dy

def transform_traj(traj, dx, dy):
    new_traj = [[p[0] + dx, p[1] + dy] for p in traj]
    return new_traj

def normalize(trajs, cellspace):
    # trajs: list of [[lon, lat], [,], ...]

    # 1. augment the input traj in order to form 2 augmented traj views
    # 2. convert augmented trajs to the trajs based on mercator space by cells
    # 3. read cell embeddings and form batch tensors (sort, pad)

    trajs1, trajs2 = [], []
    time_indices1, time_indices2 = [], []
    final_min_x = 1000000000
    final_min_y = 1000000000
    final_max_x = -1000000000
    final_max_y = -1000000000
    max_len_meters = min(cellspace.x_max-cellspace.x_min, cellspace.y_max-cellspace.y_min)
    indices_to_omit = []
    for i, l in enumerate(trajs):
        min_x, max_x, min_y, max_y = get_min_max_xy(l)
        # if final_max_x - min_x> max_len_meters or final_max_y-min_y> max_len_meters or max_x - final_min_x> max_len_meters or max_y - final_min_y> max_len_meters or max_x-min_x> max_len_meters or max_y-min_y> max_len_meters:
        #     indices_to_omit.append(i)
        #     continue
        final_min_x = min(final_min_x, min_x)
        final_min_y = min(final_min_y, min_y)
        final_max_x = max(final_max_x, max_x)
        final_max_y = max(final_max_y, max_y)
    
    if max(final_max_x-final_min_x, final_max_y-final_min_y)>max_len_meters:
        # print(max(final_max_x-final_min_x, final_max_y-final_min_y), max_len_meters)
        return None
    dx, dy = calculate_dx_dy(max_len_meters, cellspace.x_min, cellspace.y_min, final_min_x, final_max_x, final_min_y, final_max_y)
    # print(final_min_x, final_max_x, final_min_y, final_max_y)
    # print(dx, dy)
    for i,l in enumerate(trajs):
        if i in indices_to_omit:
            continue
        new_l = transform_traj(l, dx, dy)
        trajs1.append(new_l)
    return trajs1

In [20]:
from tqdm import tqdm
model.eval()
output_dict = {}

# 0: moving, 1: static, 2: not enough data
def get_gt_and_pred_label(userid):
    count = 0
    user_data = get_data_for_userid(userid)
    employers = user_data['employername'].unique()
    for employer in employers:
        emp_data = get_data_for_employername(user_data, employer)
        partitions = emp_data['partition_id'].unique()
        for partition in partitions:
            # print(userid, employer, partition)
            part_data = get_data_for_partition(emp_data, partition)
            if len(part_data)<15:
                continue
            else:
                # print(userid, employer, partition)
                # print(len(part_data))
                traj = get_traj_and_time_data(part_data)
                traj = normalize(traj, cellspace)
                # print(traj)
                if traj==None:
                    count+=1
                    pred_label = 3
                else:
                    embs = infer(traj).detach().cpu().numpy()
                    db_scan = apply_dbscan(embs, target_min_similarity=0.85)
                    labels = db_scan.labels_                      # shape: (n_samples,)
                    cluster_ids = [c for c in np.unique(labels) if c != -1]
                    if len(cluster_ids)>0:
                        pred_label = 1
                    else:
                        pred_label = 0
                output_dict[(userid, employer, partition)] = pred_label

    return count
    # if sum(test_data['paycheck_amount'].values) > 0:
    #     gt_label = 1
    # else:
    #     gt_label = 0
    # pred_label = 0
    # for i in range(len(test_embs)):
    #     test_vector = test_embs[i].unsqueeze(0)
    #     similarity = cosine_similarity(test_vector.numpy(), train_embs)[0]
    #     top_3_indices = np.argsort(similarity)[-3:][::-1]
    #     # print(i, top_3_indices)
    #     similarity = similarity[top_3_indices]
    #     # print(f"User: {userid}, Test Trajectory {test_data['traj_id'].values[i]}, Top 3 Train Trajectories: {train_data['traj_id'].values[top_3_indices]}, similarity: {similarity}, PCK Amount: {train_data['paycheck_amount'].values[top_3_indices]}")
    #     for sim, idx in zip(similarity, top_3_indices):
    #         if sim>0.85 and train_data['paycheck_amount'].values[idx]>0:
    #             pred_label = 1
    #             break
    # return gt_label, pred_label
count=0
for userid in tqdm(unique_userids):
    count+=get_gt_and_pred_label(userid)


        


100%|██████████| 1000/1000 [00:21<00:00, 46.96it/s]


In [21]:
count

71

In [22]:
# save output_dict
pickle.dump(output_dict, open("/home/sagemaker-user/TrajCL/exp/generic_v3/la_subset_output_dict.pkl", 'wb'))

In [23]:
output_dict

{(13697306, 'Maxim Healthcare Services', 1): 0,
 (13532874, 'Confie/freeway insurance', 0): 0,
 (5584592, 'Newportcare medical group', 1): 1,
 (3664893, 'Abbott', 0): 1,
 (6334226, 'rosintechnologies', 0): 3,
 (9093953, 'PepsiCo', 1): 0,
 (13867356, 'lake park communities', 0): 0,
 (13399849, 'Marriott international', 0): 0,
 (13399849, 'Marriott international', 1): 0,
 (9210700, 'Walt Disney PA', 0): 1,
 (9377174, 'COUNTY HEALTH SERVICES', 0): 0,
 (9377174, 'Maxim healthcare', 0): 0,
 (2157516, 'Amazon', 0): 3,
 (14203499, 'Curcuit services llc', 0): 1,
 (14203499, 'Industrial electronic engineers', 0): 0,
 (14475344, 'SCHARP', 0): 1,
 (10658382, '76 Gas', 0): 1,
 (15705160, 'QVC Distribution Center', 0): 0,
 (3898274, 'Kiva sales and services', 0): 1,
 (7209893, 'relativity space', 0): 0,
 (7209893, 'relativity space', 1): 0,
 (12192241, "BJ'S RESTA", 0): 1,
 (15933208, 'The gill corporation', 1): 1,
 (13342539, 'City of Los Angeles', 0): 3,
 (16690599, 'Colodro-garo llc', 0): 0,
 (1

In [24]:
count_dict = {}
for k,v in output_dict.items():
    if v in count_dict:
        count_dict[v]+=1
    else:
        count_dict[v]=1
count_dict

{0: 159, 1: 213, 3: 71}

In [25]:
def get_data_userid_employer_partition(userid, employer, partition):
    df_user = pd.concat([df[df['userid']==userid] for df in df_list]).reset_index(drop=True)
    df_emp = df_user[df_user['employername']==employer].reset_index(drop=True)
    df_part = df_emp[df_emp['partition_id']==partition].reset_index(drop=True)
    df_part_fil = df_part[~df_part['weekday'].isin([5, 6])].reset_index(drop=True) # filter out weekends
    df_part_fil = df_part_fil[df_part_fil['pck_0_1']>0].reset_index(drop=True) # filter out zero paycheck amount
    return df_part_fil

In [30]:
df_part = get_data_userid_employer_partition(3664893, 'Abbott', 0)
df_part.sort_values('traj_date')

,userid,traj_date,timestamps,polylines_final,employername,partition_id,pck_0_1,weekday,merc_seq
0,3664893,2024-09-13,"[2024-09-13T00:01:18.000000000, 2024-09-13T00:...","[[-117.90738870397912, 33.796112060546875], [-...",Abbott,0,1,4,"[[-13125390.471291533, 4001457.603220949], [-1..."
1,3664893,2024-09-17,"[2024-09-17T07:58:12.000000000, 2024-09-17T09:...","[[-118.03179962477296, 33.762420654296875], [-...",Abbott,0,1,1,"[[-13139239.831643425, 3996945.362223349], [-1..."
2,3664893,2024-09-27,"[2024-09-27T00:05:10.000000000, 2024-09-27T00:...","[[-117.90710993162298, 33.789398193359375], [-...",Abbott,0,1,4,"[[-13125359.438494798, 4000558.283135984], [-1..."
3,3664893,2024-10-01,"[2024-10-01T00:02:08.000000000, 2024-10-01T00:...","[[-117.96021522802127, 33.76605224609375], [-1...",Abbott,0,1,1,"[[-13131271.093048282, 3997431.650700413], [-1..."
4,3664893,2024-10-03,"[2024-10-03T00:04:21.000000000, 2024-10-03T00:...","[[-117.90616567586581, 33.7864990234375], [-11...",Abbott,0,1,3,"[[-13125254.32442473, 4000169.962172818], [-13..."
5,3664893,2024-10-10,"[2024-10-10T00:40:29.000000000, 2024-10-10T00:...","[[-117.8360366482587, 33.724395751953125], [-1...",Abbott,0,1,3,"[[-13117447.59678168, 3991854.87243622], [-131..."
6,3664893,2024-10-11,"[2024-10-11T00:41:21.000000000, 2024-10-11T00:...","[[-117.83387856677241, 33.740478515625], [-117...",Abbott,0,1,4,"[[-13117207.360249538, 3994007.637273137], [-1..."
7,3664893,2024-10-16,"[2024-10-16T00:40:38.000000000, 2024-10-16T00:...","[[-117.8335061461756, 33.7332763671875], [-117...",Abbott,0,1,2,"[[-13117165.902578339, 3993043.5408923957], [-..."
8,3664893,2024-10-21,"[2024-10-21T07:31:55.000000000, 2024-10-21T07:...","[[-118.01692226155897, 33.75799560546875], [-1...",Abbott,0,1,0,"[[-13137583.691146096, 3996352.8536919793], [-..."
9,3664893,2024-10-29,"[2024-10-29T08:00:26.000000000, 2024-10-29T08:...","[[-118.0282841845051, 33.758880615234375], [-1...",Abbott,0,1,1,"[[-13138848.494622892, 3996471.3529512724], [-..."


In [31]:
import numpy as np

def cosine_similarity_matrix(X, eps=1e-8, dtype=np.float32):
    X = np.asarray(X, dtype=dtype)
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    Xn = X / np.maximum(norms, eps)
    return Xn @ Xn.T

In [32]:
def get_similarity(part_df):
    traj_dates = list(part_df['traj_date'])
    traj = get_traj_and_time_data(part_df)
    traj = normalize(traj, cellspace)
    embs = infer(traj).detach().cpu().numpy()
    similarity = cosine_similarity_matrix(embs)

    return similarity, traj



In [33]:
import datetime

In [34]:
df_fil= df_part[df_part['traj_date'].isin([datetime.date(2025,3,28), datetime.date(2025,3,27)])]
sim, traj = get_similarity(df_fil)

ValueError: not enough values to unpack (expected 2, got 0)

In [37]:
sim

array([[1.        , 0.79142416],
       [0.79142416, 0.9999999 ]], dtype=float32)

In [31]:
traj

[[[4501661.205581563, 4920809.243669961],
  [4502628.720382543, 4907894.897599302],
  [4495352.767144803, 4899518.644202666],
  [4490913.6657944545, 4902065.6857565995],
  [4492585.735668652, 4905780.798577032],
  [4492443.153954195, 4911952.115215652],
  [4495100.996064605, 4913308.941421218],
  [4497705.319050238, 4915596.645665139],
  [4498934.848408513, 4917068.726972317],
  [4498965.594068378, 4919738.641298195],
  [4500467.359702548, 4920911.797075757],
  [4501727.620504184, 4921728.155148344],
  [4501382.301696956, 4921276.8947026],
  [4501390.557066074, 4920661.56838927],
  [4500913.240005737, 4921531.239298347],
  [4497710.394869184, 4921945.587025263],
  [4498311.739197807, 4921888.151785998]],
 [[4500776.996192256, 4921408.16862271],
  [4498033.472557481, 4921383.554647323],
  [4497942.899564948, 4921867.639270812],
  [4501382.642662631, 4921256.383289393],
  [4501377.118241057, 4920641.058085184],
  [4503706.222769652, 4918618.923573201],
  [4500362.515295468, 4898994.56461

In [ ]:
def meters2lonlat(x, y):
    semimajoraxis = 6378137.0
    lon = x / semimajoraxis / 0.017453292519943295
    t = math.exp(y / 3189068.5)
    lat = math.asin((t - 1) / (t + 1)) / 0.017453292519943295
    return (lon, lat)

lon_lat_1 = [meters2lonlat(x,y) for x,y in traj[0]]
lon_lat_2 = [meters2lonlat(x,y) for x,y in traj[1]]

In [ ]:
lon_lat_1

In [ ]:
lon_lat_2

In [1]:
import torch

t = torch.tensor([[1,2,3],[3,4,5]])
t.shape

torch.Size([2, 3])

In [3]:
t/t.sum(dim=0, keepdim=True)

tensor([[0.2500, 0.3333, 0.3750],
        [0.7500, 0.6667, 0.6250]])